# Explore central tendency methods

## Imports

In [58]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, unquote
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np
from scipy.stats import trim_mean



Note: you may need to restart the kernel to use updated packages.


## Get data

In [36]:
# Fetch webpage content
url = 'https://www.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/policeforceareadatatables'
response = requests.get(url)
content = response.content

# Parse the content
soup = BeautifulSoup(content, 'html.parser')

# Find all Excel file links
excel_links = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href and ('.xlsx' in href or '.xls' in href):
        full_link = f'https://www.ons.gov.uk{href}'
        excel_links.append(full_link)

# Download most recent Excel file 
if excel_links:
    excel_url = excel_links[0]  # Change the index for different files
    response = requests.get(excel_url)

   # Extract the filename from the URL
    parsed_url = urlparse(excel_url)
    filename_path = unquote(parsed_url.query)  # Decode URL-encoded characters
    filename = filename_path.split('/')[-1]  # Get the last part after the last '/'

    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"File {filename} downloaded successfully!")
else:
    print("No Excel files found.")


File pfatablessep23final.xlsx downloaded successfully!


## Prepare data

In [44]:
path = "/workspaces/practical_stats/pfatablessep23final.xlsx"
sheet_name = 'Table P1'
sheet_name_pops = 'Table P3'

# Read specific sheets and drop initial rows 
df = pd.read_excel(path, sheet_name=sheet_name, header=6)  # Assuming the first row is the header
df_pops = pd.read_excel(path, sheet_name=sheet_name_pops, header=7)

# Merge population column into crime table
df = pd.merge(df, df_pops[['Area Code', 'Population figures\n (mid-2022) rounded to 100 [note 4]']], on='Area Code', how='left')



In [60]:
# Murders and murder rate per 100,000
df_murder = df[['Area Code', 
                'Area Name', 
                'Homicide', 
                'Population figures\n (mid-2022) rounded to 100 [note 4]']].copy()

df_murder['Murder Rate'] = (df_murder['Homicide']/df_murder['Population figures\n (mid-2022) rounded to 100 [note 4]']) * 100000

# Drop rows where 'Area Code' starts with 'W', 'E12', 'E9', or 'K'
mask = (
    df_murder['Area Code'].str.startswith('W') |
    df_murder['Area Code'].str.startswith('E12') |
    df_murder['Area Code'].str.startswith('E9') |
    df_murder['Area Code'].str.startswith('K')
)

df_murder = df_murder[~mask]

# Drop row where 'Area Name' is 'British Transport Police'
df_murder = df_murder[df_murder['Area Name'] != 'British Transport Police']

## Inspect data

In [61]:
df_murder.head()

,Area Code,Area Name,Homicide,Population figures\n (mid-2022) rounded to 100 [note 4],Murder Rate
3,E23000013,Cleveland,13,579300.0,2.244088
4,E23000008,Durham,7,637600.0,1.097867
5,E23000007,Northumbria,18,1466200.0,1.227663
7,E23000006,Cheshire,5,1108800.0,0.450938
8,E23000002,Cumbria,2,503000.0,0.397614


## Central tendency

In [64]:
# Mean
pop_mean = df_murder['Population figures\n (mid-2022) rounded to 100 [note 4]'].mean()

# Trimmed mean (exclusing top and bottom 10%)
pop_trimmed_mean = trim_mean(df_murder['Population figures\n (mid-2022) rounded to 100 [note 4]'], 0.1)

# Median
pop_median = df_murder['Population figures\n (mid-2022) rounded to 100 [note 4]'].median()

print(f"""
      Population mean: {pop_mean}
      Trimmed population mean: {pop_trimmed_mean}
      Population median: {pop_median}
""")



      Mean: 1464264.1025641025
      Trimmed mean: 1251284.8484848484
      Median: 1146200.0



The mean, trimmed mean, and median provide different perspectives on the "centre" of your data, or what an "average" value looks like. 

1. **Mean (Average)**: The mean population is approximately 1,464,264. This is calculated by adding up all the populations and dividing by the number of records. It gives a general idea but can be skewed by very high or low values.
2. **Trimmed Mean**: The trimmed mean is about 1,251,285. It's another type of average, but is calculated by removing a certain percentage of the highest and lowest values before calculating the mean (in this case, the top and bottom 10%). This method can provide a more robust center point, less affected by outliers.
3. **Median**: The median population is 1,146,200. This is the middle value when all the populations are lined up from smallest to largest. Half the counties have populations above this number, and half below. The median is not influenced by extreme values in the dataset.

In summary, since the mean is higher than both the trimmed mean and the median, it suggests there might be some particularly high populations skewing the average upwards. The median gives a sense of the middle-ground population size, and the trimmed mean provides a middle value less influenced by extremes.

In [65]:
# Weighted mean
mr_weighted_mean = np.average(df_murder['Murder Rate'], weights=df_murder['Population figures\n (mid-2022) rounded to 100 [note 4]'])

# Weighted median
df_murder_sorted = df_murder.sort_values('Murder Rate')

# Calculating the cumulative sum of the weights
df_murder_sorted['cumulative_pops'] = df_murder_sorted['Population figures\n (mid-2022) rounded to 100 [note 4]'].cumsum()

# Finding the point where the cumulative population weight reaches half the total weight
total_weight = df_murder_sorted['Population figures\n (mid-2022) rounded to 100 [note 4]'].sum()
half_weight = total_weight / 2

# The weighted median is the first 'Murder Rate' value where the cumulative weight equals or exceeds half the total weight
mr_weighted_median = df_murder_sorted[df_murder_sorted['cumulative_pops'] >= half_weight]['Murder Rate'].iloc[0]

print(f"""
      Weighted murder rate mean: {mr_weighted_mean}
      Weighted murder rate median: {mr_weighted_median}
""")


      Weighted murder rate mean: 0.9946363185848146
      Weighted murder rate median: 0.9671586560699718



The weighted mean and median of the murder rate per 100,000 people provide insights into the distribution of murder rates across different areas, taking population into account. 

1. **Weighted Murder Rate Mean**: The weighted mean is approximately 0.995. This is calculated by giving more importance to murder rates from areas with larger populations. This average gives a more accurate representation of the overall murder rate, as it considers where most people live.
2. **Weighted Murder Rate Median**: The weighted median is approximately 0.967. This is the middle value of the murder rates when you arrange them in order, but like the weighted mean, it gives more importance to counties with larger populations. It represents the murder rate that splits the population into two equal groups: half living in areas with higher rates and half in areas with lower.

In summary, these weighted figures suggest that, on average, there's a murder rate of around 0.995 per 100,000 people across the counties when accounting for population size. The weighted median being slightly lower than the mean indicates that some of the more populous areas might have murder rates slightly below the weighted mean, pulling the median down.